## Лабораторная № 3. Тематическое моделирование.

*Кузьмина Н.В. J4140*

### 1. Выполните тематическое моделирование отзывов о банковских услугах помощью LDA (используйте библиотеку gensim) для одного из топ-5 банков по количеству отзывов. Для уменьшение времени вычисления можно взять выборку отзывов.

In [2]:
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# Выгрузим данные, полученные после предварительной обработки отзывов для альфа-банка:
df = pd.read_csv('alfa_bank.csv')
df

,bank_href,login,review,bank_name,site,date,message_href,score,status,user_href,preprocessed_text
0,/alfa-bank.html,NaN,История номер раз\nКредитная карта Aeroflot Go...,альфа-банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN,история номер кредитный карта aeroflot gold об...
1,/alfa-bank.html,NaN,"Отличный банк Хорошие депозиты, высокие процен...",альфа-банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN,отличный банк хороший депозит высокий процент ...
2,/alfa-bank.html,NaN,Только что с жену ограбили во время съёма дене...,альфа-банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN,жена ограбить время съём деньга банкомат допол...
3,/alfa-bank.html,NaN,Заполнила заявку на получение дебетовой карты ...,альфа-банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN,заполнить заявка получение дебетовый карта nex...
4,/alfa-bank.html,NaN,О сервисе Альфа-клик думаю плохо. Очень плохо....,альфа-банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN,сервис альфа-клик думать плохо очень плохо поя...
...,...,...,...,...,...,...,...,...,...,...,...
1255,/bank/alfa-bank/,"Юрий,","Прямо приятно было деньги отдавать :-) Нет, ну...",альфа-банк,https://www.sravni.ru,"19 окт, 2015",/bank/alfa-bank/otzyv/178722/?page,5,NaN,NaN,прямо приятно деньга отдавать правда случиться...
1256,/bank/alfa-bank/,"Павел,",Хороший банкХороший банк,альфа-банк,https://www.sravni.ru,"20 окт, 2015",/bank/alfa-bank/otzyv/83/?page,2,NaN,NaN,хороший банкхороший банк
1257,/bank/alfa-bank/,"Анна Мордовченко,",Общение с клиентами дистанционноОформила креди...,альфа-банк,https://www.sravni.ru,"30 сент, 2018",/bank/alfa-bank/otzyv/299285/?page,3,NaN,NaN,общение клиент дистанционнооформить кредит тел...
1258,/bank/alfa-bank/,"оксана,",Нарушены все сроки рассмотрения заявок на ипот...,альфа-банк,https://www.sravni.ru,"03 окт, 2018",/bank/alfa-bank/otzyv/299519/?page,1,NaN,NaN,нарушить срок рассмотрение заявка ипотека пода...


In [4]:
# Преобразуем значения "Без оценки" в NaN
df['score'] = df['score'].replace('Без оценки', np.nan)

# Преобразуем значения в столбце 'score' в числовой тип данных
df['score'] = pd.to_numeric(df['score'], errors='coerce')

# Вычисляем среднее значение столбца 'score'
mean_score = df['score'].mean()

# Удаляем все строки, содержащие NaN
df = df.loc[~df['score'].isna()]

In [5]:
df.shape

(885, 11)

In [6]:
data = df.preprocessed_text.values.tolist()

In [8]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [9]:
# tokenize
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[0])
print(len(data_words))

['являться', 'постоянныи', 'клиент', 'альфа', 'банк', 'год', 'иметь', 'несколько', 'полностью', 'выплатить', 'кредит', 'данныи', 'банк', 'являться', 'зарплата', 'клиентом', 'ть', 'январь', 'заполнить', 'саит', 'банк', 'заявка', 'кредит', 'продукт', 'одобрить', 'предложить', 'приити', 'офис', 'оформление', 'сожаление', 'добавление', 'саит', 'кредитныи', 'карта', 'произоити', 'сбои', 'учесть', 'сумма', 'кредит', 'наличными', 'по', 'телефон', 'сообщить', 'проблема', 'необходимо', 'обратиться', 'офис', 'корректировка', 'заявки', 'январь', 'обратиться', 'офис', 'внести', 'корректировка', 'заявка', 'одобрить', 'момент', 'начаться', 'чудо', 'сотрудник', 'банк', 'сказать', 'кредит', 'одобрить', 'заключение', 'договор', 'страхование', 'год', 'срок', 'ич', 'вопрос', 'указывать', 'заявка', 'необходимыи', 'страховка', 'корректировка', 'также', 'задавать', 'данныи', 'вопрос', 'сотрудник', 'ответить', 'решение', 'москва', 'мочь', 'повлиять', 'ть', 'спросить', 'отказаться', 'страховка', 'подчеркивать

In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=100) # higher threshold =>> fewer number of bigrams.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['являться', 'постоянныи', 'клиент', 'альфа', 'банк', 'год', 'иметь', 'несколько', 'полностью', 'выплатить', 'кредит', 'данныи', 'банк', 'являться', 'зарплата', 'клиентом', 'ть', 'январь', 'заполнить', 'саит', 'банк', 'заявка', 'кредит', 'продукт', 'одобрить', 'предложить', 'приити', 'офис', 'оформление', 'сожаление', 'добавление', 'саит', 'кредитныи', 'карта', 'произоити', 'сбои', 'учесть', 'сумма', 'кредит', 'наличными', 'по', 'телефон', 'сообщить', 'проблема', 'необходимо', 'обратиться', 'офис', 'корректировка', 'заявки', 'январь', 'обратиться', 'офис', 'внести', 'корректировка', 'заявка', 'одобрить', 'момент', 'начаться', 'чудо', 'сотрудник', 'банк', 'сказать', 'кредит', 'одобрить', 'заключение', 'договор', 'страхование', 'год', 'срок', 'ич', 'вопрос', 'указывать', 'заявка', 'необходимыи', 'страховка', 'корректировка', 'также', 'задавать', 'данныи', 'вопрос', 'сотрудник', 'ответить', 'решение', 'москва', 'мочь', 'повлиять', 'ть', 'спросить', 'отказаться', 'страховка', 'подчеркивать

In [11]:
print(bigram_mod[data_words[1]])

['день', 'добрыи', 'мыть', 'мама', 'пенсионер', 'хотеть', 'оформить', 'кредит', 'приехать', 'офис', 'затратить', 'дорога', 'менее', 'час', 'просидеть', 'течение', 'час', 'заполнять', 'анкета', 'хотя', 'анкета', 'заполнить', 'сутки', 'ранее', 'саит', 'алф', 'банк', 'предодобрить', 'сказать', 'банк', 'одобрить', 'железно', 'попросить', 'подождать', 'некоторыи', 'время', 'проверка', 'анкета', 'андераитер', 'час', 'поступить', 'звонок', 'банк', 'спрашивать', 'анкета', 'еметься', 'кредит', 'стороннии', 'банк', 'мина', 'звонок', 'приити', 'смс', 'банк', 'мочь', 'настоящии', 'время', 'выдать', 'кредит', 'шооокии', 'сотрудник', 'лицо', 'банк', 'говорить', 'одобрить', 'ноль', 'цена', 'скорее', 'лицензия', 'лишить', 'альфа', 'банк', 'банк', 'отвечать', 'слово']


In [12]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [13]:
data_words_bigrams = make_bigrams(data_words)

In [14]:
# Преобразование текста в формат, понятный Gensim
texts = data_words_bigrams
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

In [15]:
print(corpus[:1])

[[(0, 1), (1, 8), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 2), (10, 4), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 4), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 3), (37, 4), (38, 3), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 2), (52, 4), (53, 1), (54, 2), (55, 1), (56, 1), (57, 1), (58, 3), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 3), (77, 1), (78, 1), (79, 1), (80, 2), (81, 2), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 5), (90, 1), (91, 1), (92, 4), (93, 4), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 2), (101, 1), (102, 1), (103, 1), (104, 1), (105, 3), (106, 2)]]


In [16]:
print(corpus[1])

[(0, 1), (1, 8), (25, 1), (37, 3), (42, 1), (52, 2), (58, 1), (73, 1), (81, 1), (85, 1), (89, 1), (107, 1), (108, 1), (109, 4), (110, 2), (111, 1), (112, 1), (113, 1), (114, 1), (115, 1), (116, 1), (117, 1), (118, 1), (119, 1), (120, 2), (121, 1), (122, 1), (123, 1), (124, 1), (125, 1), (126, 1), (127, 1), (128, 1), (129, 1), (130, 1), (131, 1), (132, 1), (133, 1), (134, 1), (135, 1), (136, 1), (137, 1), (138, 1), (139, 1), (140, 1), (141, 1), (142, 1), (143, 1), (144, 1), (145, 1), (146, 1), (147, 1), (148, 1), (149, 1), (150, 1), (151, 1), (152, 3), (153, 1)]


In [17]:
len(corpus)

885

In [18]:
[[(id2word[id], freq) for id, freq in enumerate(cp)] for cp in corpus[:1]]

[[('альфа', (0, 1)),
  ('банк', (1, 8)),
  ('брать', (2, 1)),
  ('вид', (3, 1)),
  ('видимо', (4, 1)),
  ('включить', (5, 1)),
  ('внести', (6, 1)),
  ('вопрос', (7, 3)),
  ('выплатить', (8, 1)),
  ('год', (9, 2)),
  ('данныи', (10, 4)),
  ('дать', (11, 1)),
  ('деиствие', (12, 1)),
  ('добавление', (13, 1)),
  ('добровольныи', (14, 1)),
  ('договор', (15, 1)),
  ('дорогои', (16, 1)),
  ('забивать', (17, 1)),
  ('зависеть', (18, 1)),
  ('задавать', (19, 1)),
  ('задать', (20, 1)),
  ('заключение', (21, 1)),
  ('закона', (22, 1)),
  ('заменить', (23, 1)),
  ('заново', (24, 1)),
  ('заполнить', (25, 1)),
  ('зарплата', (26, 1)),
  ('заявка', (27, 4)),
  ('заявки', (28, 1)),
  ('иии', (29, 1)),
  ('иметь', (30, 1)),
  ('исключение', (31, 1)),
  ('ич', (32, 1)),
  ('карта', (33, 1)),
  ('клиент', (34, 2)),
  ('клиентом', (35, 1)),
  ('корректировка', (36, 3)),
  ('кредит', (37, 4)),
  ('кредитныи', (38, 3)),
  ('менять', (39, 1)),
  ('момент', (40, 1)),
  ('москва', (41, 2)),
  ('мочь', (4

In [19]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [20]:
from pprint import pprint

In [21]:
# Выведем 15 первых тем с ключевыми словами
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.040*"кредит" + 0.040*"банк" + 0.024*"звонить" + 0.024*"звонок" + '
  '0.021*"номер" + 0.020*"сотрудник" + 0.014*"мочь" + 0.014*"телефон" + '
  '0.013*"альфа" + 0.013*"заявление"'),
 (1,
  '0.019*"дарья" + 0.016*"баланс" + 0.014*"трудность" + 0.012*"директ" + '
  '0.010*"поблагодарить" + 0.008*"частныи" + 0.006*"индивидуальныи_подход" + '
  '0.004*"вывести" + 0.003*"автоматическии" + 0.002*"убеждаться"'),
 (2,
  '0.028*"контактныи" + 0.021*"нарушение" + 0.019*"персональныи" + '
  '0.019*"заявить" + 0.015*"неизвестныи" + 0.015*"фз" + 0.014*"незаконныи" + '
  '0.013*"сентинела" + 0.011*"александрович" + 0.011*"персональныи_данные"'),
 (3,
  '0.020*"месяц" + 0.019*"названивать" + 0.015*"работник" + 0.014*"начислить" '
  '+ 0.012*"тратить" + 0.012*"машина" + 0.011*"измениться" + 0.011*"кешбек" + '
  '0.011*"здоровье" + 0.011*"миля"'),
 (4,
  '0.061*"льготныи_период" + 0.017*"ст" + 0.016*"неожиданно" + '
  '0.010*"оперативность" + 0.009*"добросовестныи" + 0.008*"инициатива" + '
  '

### 2. Найдите оптимальное количество тем в соответствии с метриками: 1) Perplexity score 2) Coherence score. Есть ли различия?

In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.754116731872047

Coherence Score:  0.4520725650415251


Различия между этими метриками заключаются в том, что перплексия фокусируется на качестве прогнозирования новых данных, в то время как когерентность оценивает интерпретируемость тем.

1. Отрицательное значение перплексии говорит о том, что модель хорошо предсказывает новые данные.
2. Чем выше значение когерентности, тем лучше модель. В данном случае значение когерентности показывает, что темы, выделенные моделью, имеют хорошую связь между собой и интерпретируемы.


### 3. Как бы вы назвали каждую тему? Обоснуйте.

Для названия каждой темы в модели LDA использовались наиболее часто встречающиеся слова (токены) в каждой теме. Эти слова служат ключевыми терминами для определения содержания каждой темы.

Названия для первых 11 топиков, которые были выведены выше:

* 0: "Проблемы с кредитом и обслуживанием в банке" - данная тема содержит ключевые слова, связанные с кредитами, звонками в банк, номерами, заявлениями и сотрудниками банка.
* 1:  "Благодарность за индивидуальный подход" - ключевые слова в этой теме указывают на благодарность конкретному сотруднику или отделу банка за индивидуальный подход к клиентам.
* 2: "Нарушения в обработке персональных данных"
* 3: "Транзакции и начисления" - ключевые слова в этой теме указывают на месячные начисления, траты, кешбек и другие финансовые операции.
* 4: "Льготный период и законодательство" - данная тема связана с льготным периодом по кредитам и законодательством в этой области.
* 5: "Операции со счетами" - ключевые слова указывают на операции со счетами, переводы, претензии и обращения клиентов в банк.
* 6: "Различные ситуации и запросы клиентов" - данная тема содержит ключевые слова, связанные с различными ситуациями, запросами и обращениями клиентов.
* 7: "Развитие карьеры и оперативность" - ключевые слова указывают на развитие карьеры, оперативность и рост в банковской сфере.
* 8: "Кредитные карты и услуги банка" - данная тема содержит ключевые слова, связанные с банковскими картами, клиентами, комиссиями и кредитами.
* 9: "Путешествия и приоритетное обслуживание" - ключевые слова указывают на путешествия, приоритетное обслуживание и различные услуги для клиентов.
* 10: "Ипотека и профессионализм менеджеров" - данная тема связана с ипотекой, ставками, документами

### 4. Визуализируйте темы с помощью пакета pyLDAvis.

In [25]:
# !pip install pyLDAvis

In [26]:
# LDAvis
import pyLDAvis
import pyLDAvis.gensim_models

In [27]:
# Визуализация тем
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

### 5. Связаны ли темы отзывовов и поле “Score” в данных? Обоснуйте.